# 04: Point Localization

*Authors: Lars Nitzschke, Prof. Dr. Kevin Buchin*

This notebook serves as supplementary learning material for the course **Geometric Algorithms**.
It showcases and explains implementations of algorithms presented in the corresponding lecture, and elaborates on some practical considerations concerning their use.
Furthermore, it offers interactive visualisations and animations.

## Table of Contents

1. Introduction  
2. Setup  
3. Data Structures    
    3.1. Slab Decomposition  
    3.2. Vertical Decomposition
4. References  

## 1. Introduction

We start with the definition of the **point localization problem**. In the lecture it was stated as follows: Preprocess a planar subdivision such that for any query point _q_, the face of the subdivision containing _q_ can be given efficiently.
A **planar subdivision** is a partition of the plane by a a sert of non-crossing line segments into vertices edges and faces. The question at hand is a data structuring question, so we are interested in the **query time**, the **storage requirements** and the **preprocessing time**.

See the following two images for examples. They show the slab decomposition and the vertical decomposition respectively (on slightly different instances). The input **doubly-connected edge list** is coloured <font color='orange'>orange</font>, while the slab / vertical decomposition is marked in <font color='blue'>blue</font>.

<img style='float: left;' src='./images/04-image00.png' width='440px' height='412'>
<img style='float: left;' src='./images/04-image01.png' width='440px' height='412'>

## 2. Setup

First let's do some setup. This is not very interesting, so you can skip to Section 3 if you want.

We now import everything we'll need throughout this notebook from external sources, including our module for generic data structures, our module for geometric primitives and operations as well as our module for visualisation purposes. 
These modules are explained in [notebook no. 00](./00-Basics.ipynb).

In [ ]:
# Python standard library imports
import math
from typing import Any, Optional
from itertools import combinations, chain

# Data structure, geometry and visualisation module imports
from modules.data_structures import BinaryTree, BinaryTreeDict, Comparator, ComparisonResult as CR, DoublyConnectedSimplePolygon, HalfEdge, DoublyConnectedEdgeList, PointLocation, Face, VDLineSegment, AnimationBinaryTreeDict, PLSearchStructure
from modules.geometry import Point, PointReference, PointSequence, LineSegment, Orientation as ORT, EPSILON, Rectangle
from modules.visualisation import VisualisationTool, PointLocationInstance, SlabDecompositionMode, VerticalExtensionMode, PointLocationMode

Additionally, we create an object for our visualisation tool and register a few example instances.

In [ ]:
visualisation_tool = VisualisationTool(400, 400, PointLocationInstance(drawing_epsilon=10))
canvas_size = min(visualisation_tool.width, visualisation_tool.height)

c = 0.5 * canvas_size
r = 0.9 * c
s = c - r
t = c + r
u = (t - s) / 36

points = [Point(s + 20*u, t -  1*u), Point(s + 25*u, t -  6*u), Point(s + 30*u, t -  9*u), Point(s + 28*u, t - 11*u),
          Point(s + 19*u, t - 14*u), Point(s + 30*u, t - 21*u), Point(s + 28*u, t - 29*u), Point(s + 20*u, t - 33*u),
          Point(s + 15*u, t - 31*u), Point(s + 11*u, t - 34*u), Point(s +  2*u, t - 30*u), Point(s + 11*u, t - 24*u),
          Point(s +  5*u, t - 22*u), Point(s +  2*u, t - 17*u), Point(s +  5*u, t - 14*u), Point(s +  5*u, t -  6*u),
          Point(s + 11*u, t -  3*u), Point(s + 15*u, t - 17*u), Point(s + 19*u, t - 27*u)]
          
edges = [(0,1), (1,2), (2,3), (3,4), (4,5), (5,6), (6,7), (7,8), (8,9), (9,10), (10,11), (11,12), (12,13), (13,14),
         (14,15), (15,16), (16,0), (1,3), (4,16), (4,17), (12,17), (14,17), (5,11), (6,18), (10,18)]

dcel = DoublyConnectedEdgeList(points, edges)
visualisation_tool.register_example_instance("Course Example 1", dcel)

points = [Point(s + 20*u, t -  1*u), Point(s + 25*u, t -  6*u), Point(s + 31*u, t -  9*u), Point(s + 28*u, t - 11*u),
          Point(s + 19*u, t - 14*u), Point(s + 30*u, t - 21*u), Point(s + 27*u, t - 29*u), Point(s + 22*u, t - 33*u),
          Point(s + 13*u, t - 31*u), Point(s +  8*u, t - 34*u), Point(s +  2*u, t - 30*u), Point(s + 11*u, t - 24*u),
          Point(s +  5*u, t - 22*u), Point(s +  1*u, t - 17*u), Point(s +  4*u, t - 14*u), Point(s +  6*u, t -  6*u),
          Point(s + 10*u, t -  3*u), Point(s + 15*u, t - 17*u), Point(s + 18*u, t - 27*u)]

dcel = DoublyConnectedEdgeList(points, edges)
visualisation_tool.register_example_instance("Course Example 2", dcel)

visualisation_tool.display()

## 3. Algorithms

In this notebook the first take a look at the simple **slab decomposition** and afterwards we approach the more sophisticated **vertical decomposition**.

### 3.1 Slab Decomposition

The first step in the lecture towards the **vertical decompostion** was the **slab decomposition**. Essentially the problem is reduced to a one-dimensional problem, which can efficiently be solved using a balanced binary search tree. To reduce the problem we draw vertical lines through all vertices, which divides the instance into a set of vertical strips, also called slabs. Inside each strip there is a well-defined _bottom-to-top order_ on the line segments. Each strip is therefore a one dimensional problem in _y_-direction. If we know between which line segments of the strip our point is, we found the face of the subdivision containing the point. Finding the correct strip is also a one dimensional problem, this time in _x_-direction.

Both one-dimensional problems are solved using a binary tree. (More information on the used binary trees can be found in [notebook no. 00](./00-Basics.ipynb).)
We start with the comparators for our trees. Same as in the previous notebooks we use a small epsilon in the comparison to somewhat compensate for the inaccuracy of floating-point arithmetic. (See [notebook no. 00](./00-Basics.ipynb) for further information on this as well.)

In [ ]:
class PointXComparator(Comparator[Point]):
    def compare(self, item: Any, key: Point) -> CR:
        if not isinstance(item, Point):
            raise TypeError("This comparator can only compared points to points")
        elif abs(item.x - key.x) < EPSILON:
            return CR.MATCH
        elif item.x - key.x > EPSILON:
            return CR.AFTER
        else:
            return CR.BEFORE
        
class LineSegmentYComparator(Comparator[LineSegment]):
    def compare(self, item: Any, key: LineSegment) -> CR:
        if isinstance(item, Point):
            if abs(item.y - key.y_from_x(item.x)) < EPSILON:
                return CR.MATCH
            elif item.y - key.y_from_x(item.x) > EPSILON:
                return CR.AFTER
            else:
                return CR.BEFORE
        elif isinstance(item, LineSegment):
            if abs(item.y_from_x(key.left.x) - key.left.y) < EPSILON and abs(item.y_from_x(key.right.x) - key.right.y) < EPSILON:
                return CR.MATCH
            elif item.left.y - key.left.y > EPSILON \
                or abs(item.y_from_x(key.left.x) - key.left.y) < EPSILON and item.y_from_x(key.right.x) - key.right.y > EPSILON:
                return CR.AFTER
            else:
                return CR.BEFORE        
        else:
            raise TypeError("This comparator can only compare either points or line segments to line segments")

Using these comparators our data structure is constructed as follows. We build a <tt>AnimationBinaryTreeDict</tt> $\mathcal{T}$ on all points in _x_-order. With each point we store a subtree on the line segments crossing the slab left of the point in _y_-order. This subtree is a <tt>AnimationBinaryTreeDict</tt> as well and with each line segement we store the face lying directly above. Note that in [1, section 6.1] the points are stored in an array instead. This is equivalent when using binary search on the array.

The line segments crossing each slab are computed using the sweep line technique. At each point we regard the linesegments which start and end here (left to right) and keep track of all currently crossing line segments. In the degenerate case where two or more points have the same _x_-coordinate we do compute our currently crossing line segments at each point, but only add the last point of that _x_-coordinate to $\mathcal{T}$.

In [ ]:
class SlabDecomposition(PLSearchStructure):
    def __init__(self, dcel: DoublyConnectedEdgeList, bounding_box: Rectangle, epsilon = EPSILON):
        # Stuff for drawing the vertical extensions at each point
        self._bounding_box = bounding_box
        self._point_sequence = PointSequence()
        # The outer face of the dcel is necessary when a search point does not lie between two line segments.
        self._ubounded_face = dcel.outer_face
        # Our actual tree structure
        self._x_comparator = PointXComparator()
        self._y_comparator = LineSegmentYComparator()
        self.search_tree: AnimationBinaryTreeDict[Point, AnimationBinaryTreeDict[LineSegment, Face]] = AnimationBinaryTreeDict(self._x_comparator, lambda p: p)

        currently_crossing_edges: list[HalfEdge] = []  # TODO: Maybe make this a tree.

        sorted_points = sorted(dcel.vertices, key = lambda vertex: vertex.point.x)
        iterator = iter(sorted_points)
        next_vertex = next(iterator, None)

        while next_vertex is not None:
            vertex = next_vertex
            next_vertex = next(iterator, None)

            if vertex.outgoing_edges() == []:  # Single vertices (i.e. no outgoing edges) are ignored as they lie completely inside one face.
                continue
            
            # Keep track of currently crossing line segments
            currently_crossing_edges = list(filter(lambda edge: edge.right.x - vertex.x > epsilon, currently_crossing_edges))
            currently_crossing_edges.extend(filter(lambda edge: edge.right.x - vertex.x > epsilon, vertex.outgoing_edges()))

            if next_vertex is not None and abs(vertex.x - next_vertex.x) < epsilon:  # Degenerate case: same x-coordinate
                self._point_sequence.append(vertex.point)
                continue

            # Build tree on the currently crossing line segments in y-direction
            subtree = AnimationBinaryTreeDict(self._y_comparator, lambda ls: PointReference([ls.left, ls.right], 0))
            for edge in currently_crossing_edges:
                subtree.insert(LineSegment(edge.left, edge.right), edge.incident_face)

            self.search_tree.insert(vertex.point, subtree)
            self._point_sequence.append(PointReference([vertex.point, Point(vertex.point.x, self._bounding_box.lower), Point(vertex.point.x, self._bounding_box.upper)], 0))

    def query(self, point: Point) -> tuple[Face, PointSequence]:
        ps = PointSequence()
        ps.append(point)
        # Search in x direction for the correct slab
        x_pred, x_ps = self.search_tree.search_predecessor(point)
        ps = ps + x_ps
        if x_pred is None:  # The search point is in the unbounded slab on the left side
            ps.clear()
            ps.append(point)
            return self._ubounded_face, ps
        else:
            y_tree = x_pred[1]
            # Search inside the slab for the correct face
            y_pred, y_ps = y_tree.search_predecessor(point)
            ps = ps + y_ps
            if y_pred is None:  # The search point is in the unbounded slab on the right side or below the first line segment in this slab.
                ps.clear()
                ps.append(point)
                return self._ubounded_face, ps
            dcel_face = y_pred[1]
            
            ps.clear()
            ps.append(point)
            for point in dcel_face.outer_points():
                ps.append(point)
            return dcel_face, ps

The storage requirements of the **slab decompostion** is $O(n^2)$ spacem where $n$ is the number of line segments. We have a tree of size $O(n)$, as we could store all points in the tree. With each point we store a subtree of size $O(n)$ because we store the line segments crossing the slab. See the lecture slides or [1, section 6.1] for an example using worst case storage.

Querying a point in our finished data structure then works as follows: Firstly we search in _x_-direction for the predecessor of our search point in our tree $\mathcal{T}$.
The subtree $\mathcal{T}_{y}$ stored with this predecessor represents the slab lying to its right. If we do not find a predecessor our search point lies in the far left unbounded face.
The outer dcel face is returned. Secondly we search for the predecessor of our search point in the subtree $\mathcal{T}_{y}$. The resulting line segment lies directly below the search point.
The face stored with the line segment is the face containing our search point. Thus we return the face.
If our search in the subtree $\mathcal{T}_{y}$ does not return a line segment our search point lies either in the unbounded face on the right or below the first line segment of that slab.
Again, we return the outer dcel face.

The query time for the data structure is good: We perform a search in the main tree $\mathcal{T}$ and a search in the subtree $\mathcal{T}_{y}$. Hence the query time is $O(\text{log}\; n)$.

Let's register both the construction of our slab decomposition and the point location query for visualisation.

In [ ]:
# Construction of the slab decomposition
def build_slab_decompostion(pl: PointLocation) -> PointSequence:
    pl._search_structure = SlabDecomposition(pl._dcel, bounding_box = Rectangle(Point(0, 0), Point(visualisation_tool._width, visualisation_tool._height)))
    return pl._search_structure._point_sequence

visualisation_tool.register_algorithm("Build Slab Decomp.", build_slab_decompostion, VerticalExtensionMode())

# Point location in the slab decompostion
def slab_point_location(pl: PointLocation) -> PointSequence:
    # Get search point from dcel
    point = pl._dcel._last_added_vertex.point
    point_edges = pl._dcel.find_edges_of_vertex(pl._dcel._last_added_vertex)
    if len(point_edges) > 1 or point_edges[0].origin != point_edges[0].destination:
        raise ValueError(f"Point Location needs to search for a single point not connected to any other point.")
    # Query the point
    face, point_sequence = pl._search_structure.query(point)
    return point_sequence

visualisation_tool.register_algorithm("Slab Point Location", slab_point_location, PointLocationMode(), preprocessing=build_slab_decompostion)

Now you can interactively test the algorithm and watch animations of it!
You need to run the notebook cells up until the following one for this purpose.
If you haven't used our interactive visualisation tool before, see [notebook no. 00](./00-Basics.ipynb) for an explanation.

For the the slab point location you need to add a search point to the instance by clicking the canvas. The query will always use the latest added point.
You can add edges to the instance by clicking an existing vertex as the first endpoint and for the second endpoint you can either click another existing vertex or create a new vertex by clicking clicking the canvas.

In [ ]:
visualisation_tool.display()

__*Takeaway:*__

* The slab decomposition is easy to implement and has a good query time, but its storage requirements make it a rather uninteresting data structure.

### 3.2 Vertical Decomposition

In [ ]:
# Construction of the vertical decomposition
def build_vertical_decomposition(pl: PointLocation) -> PointSequence:
    pl.build_vertical_decomposition(pl.dcel_prepocessing(pl._dcel), random_seed = 42)
    return pl._vertical_decomposition._point_sequence

visualisation_tool.register_algorithm("Build Vert. Decomp.", build_vertical_decomposition, VerticalExtensionMode())

# Point location in the vertical decomposition
def point_location(pl: PointLocation) -> PointSequence:
    point = pl._dcel._last_added_vertex.point
    point_edges = pl._dcel.find_edges_of_vertex(pl._dcel._last_added_vertex)
    if len(point_edges) > 1 or point_edges[0].origin != point_edges[0].destination:
        raise ValueError(f"Point Location needs to search for a single point not connected to any other point.")
    dcel_face, ps = pl._search_structure.query(point)
    return ps

visualisation_tool.register_algorithm("Point Location", point_location, PointLocationMode(), preprocessing=build_vertical_decomposition)

# Display visualisation tool
visualisation_tool.display()

In [ ]:
# TODO: Animation of DCEL drawings

# TODO: DCEL check for crossings and raise exception or add as segment not as an edge
# TODO: Clicking the canvas sometimes does not work

# TODO: Add AnimationTrees and DCEL to notebook 00.
# TODO: Look into ETE Toolkit for tree visualisation

# TODO: Move code of the vertical decompostion from the module to the notebook and write explanations
# TODO: Reset execution counter of cells and add images as default cell outputs.